In [1]:
from google.colab import drive
drive.mount('/gdrive')

Mounted at /gdrive


In [2]:
import pandas as pd
import numpy as np

# 1. 데이터 불러오기
1. 모델학습으로 만든 제출 데이터
2. TF-IDF 1번방식으로 만든 덮어쓰기 파일
3. TF-IDF 2번방식으로 만든 덮어쓰기 파일

In [14]:
sub = pd.read_csv('/gdrive/MyDrive/2조/data/코드정리/Submission/sub54_VC.csv')
cover1= pd.read_csv('/gdrive/MyDrive/2조/data/코드정리/TF-IDF로 찾은 덮어쓰기/최소유사도 0.7이상이고 전체에 68프로가 해당되는 Knowcode 덮어쓰기.csv')
cover2= pd.read_csv('/gdrive/MyDrive/2조/data/코드정리/TF-IDF로 찾은 덮어쓰기/최소유사도 0.4이상이고 최대로부터 0.3아래 중 전체에 68프로가 해당되는 Knowcode 덮어쓰기.csv')

# 2. 합치기

In [15]:
cover_total = pd.merge(cover1,cover2,how='outer',on = 'idx')
cover_total

,idx,knowcode_x,knowcode_y
0,6,232101.0,232101.0
1,13,155305.0,NaN
2,14,301004.0,301004.0
3,19,881101.0,881101.0
4,20,541301.0,541301.0
...,...,...,...
8622,35033,NaN,29401.0
8623,35058,NaN,32101.0
8624,35157,NaN,153301.0
8625,35181,NaN,32101.0


합치면서 생긴 부분을 0으로 채워주고 최종적으로 덮어쓴값들을 저장해줄 컬럼 생성

In [16]:
cover_total = cover_total.fillna(0)
cover_total.columns = ['idx','knowcode_x','knowcode_y']
cover_total['knowcode'] = np.nan
cover_total

,idx,knowcode_x,knowcode_y,knowcode
0,6,232101.0,232101.0,NaN
1,13,155305.0,0.0,NaN
2,14,301004.0,301004.0,NaN
3,19,881101.0,881101.0,NaN
4,20,541301.0,541301.0,NaN
...,...,...,...,...
8622,35033,0.0,29401.0,NaN
8623,35058,0.0,32101.0,NaN
8624,35157,0.0,153301.0,NaN
8625,35181,0.0,32101.0,NaN


## 2-1. 중복되는 답만 살리기

In [17]:
#중복되는 답만 살리기
for i in range(len(cover_total)):
  if cover_total['knowcode_x'].iloc[i] == cover_total['knowcode_y'].iloc[i]:
    cover_total['knowcode'].iloc[i] = cover_total['knowcode_y'].iloc[i]

/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


In [18]:
cover_total = cover_total[['idx','knowcode']].dropna()
cover_total = cover_total[cover_total['knowcode'] != 0]

In [20]:
cover_total

,idx,knowcode
0,6,232101.0
2,14,301004.0
3,19,881101.0
4,20,541301.0
5,26,550201.0
...,...,...
8026,35231,811201.0
8027,35235,702201.0
8029,35240,155303.0
8030,35241,29401.0


중복되는 답을 모은것을 저장

In [21]:
cover_total.to_csv('/gdrive/MyDrive/2조/data/코드정리/TF-IDF로 찾은 덮어쓰기/sub_더많은 키워드_유사도및겹치기0.7_최초0.4최대로부터0.3겹치는거0.7_두개 중복답.csv',index = False)

모델 학습으로 뽑은 제출파일과 합치기

In [22]:
new_sub = pd.merge(sub, cover_total, how = 'left',on = 'idx')
new_sub.columns = ['idx','knowcode','newcode']
new_sub = new_sub.fillna(0)
new_sub

,idx,knowcode,newcode
0,0,29401,0.0
1,1,415101,0.0
2,2,412001,0.0
3,3,121102,0.0
4,4,412003,0.0
...,...,...,...
35226,35244,811101,833001.0
35227,35245,29303,0.0
35228,35246,833001,0.0
35229,35247,833001,0.0


In [23]:
#newcode에서 변환하기위해 준비된 값들을 제출용 knowcode로 덮어쓰기함
for i in range(len(new_sub)):
  if new_sub['newcode'].iloc[i] !=  0:
    new_sub['knowcode'].iloc[i] = int(new_sub['newcode'].iloc[i])

new_sub

/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


,idx,knowcode,newcode
0,0,29401,0.0
1,1,415101,0.0
2,2,412001,0.0
3,3,121102,0.0
4,4,412003,0.0
...,...,...,...
35226,35244,833001,833001.0
35227,35245,29303,0.0
35228,35246,833001,0.0
35229,35247,833001,0.0


In [25]:
sub_final = new_sub[['idx','knowcode']]
sub_final.to_csv('/gdrive/MyDrive/2조/data/코드정리/Submission/sub54_VC에 덮어쓰기 두개 중복답.csv',index = False)